In [ ]:
import numpy as np #vector and matrices
import pandas as pd #tables and data manipulation
import matplotlib.pyplot as plt #plots
plt.rcParams['figure.figsize'] = 20, 15
plt.style.use('seaborn')
import statsmodels.tsa.seasonal as sm #statistics
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

## Exploración de los datos y carga del DF


In [2]:
# Importar el csv
df = pd.read_csv('./avocado.csv').copy() 

# Verificar la carga
df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
dtypes: float64(9), int64(2), object(3)
memory usage: 1.9+ MB


In [4]:
df.describe()

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,18249.000000,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,24.232232,1.405978,8.506440e+05,2.930084e+05,2.951546e+05,2.283974e+04,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,15.481045,0.402677,3.453545e+06,1.264989e+06,1.204120e+06,1.074641e+05,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,0.000000,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,10.000000,1.100000,1.083858e+04,8.540700e+02,3.008780e+03,0.000000e+00,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,24.000000,1.370000,1.073768e+05,8.645300e+03,2.906102e+04,1.849900e+02,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,38.000000,1.660000,4.329623e+05,1.110202e+05,1.502069e+05,6.243420e+03,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,52.000000,3.250000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [5]:
df.count()

Unnamed: 0      18249
Date            18249
AveragePrice    18249
Total Volume    18249
4046            18249
4225            18249
4770            18249
Total Bags      18249
Small Bags      18249
Large Bags      18249
XLarge Bags     18249
type            18249
year            18249
region          18249
dtype: int64

In [6]:
unique_values = df.nunique()
unique_values

Unnamed: 0         53
Date              169
AveragePrice      259
Total Volume    18237
4046            17702
4225            18103
4770            12071
Total Bags      18097
Small Bags      17321
Large Bags      15082
XLarge Bags      5588
type                2
year                4
region             54
dtype: int64

In [7]:
df['AveragePrice'].shape

(18249,)

## Etiquetar cada region según su tipo

El objetivo es lograr categorizar la columna region en tres tipos principales, según sea 'cities', 'regions', 'greatregions'

In [8]:
# Crear nueva columna region_type para designar una categoría a cada region

# Listas para la clasificación de regiones
cities = ['Albany', 'Atlanta', 'Boise', 'Boston', 'Charlotte', 'Chicago', 'Columbus', 'Denver', 
          'Detroit', 'GrandRapids', 'Houston', 'Indianapolis', 'Jacksonville', 'LasVegas', 'LosAngeles',
          'Louisville', 'Nashville', 'NewYork', 'Orlando', 'Philadelphia', 'Pittsburgh', 'Portland', 
          'Roanoke', 'Sacramento', 'SanDiego', 'SanFrancisco', 'Seattle', 'Spokane', 'StLouis', 'Syracuse', 'Tampa']

regions = ['BaltimoreWashington', 'BuffaloRochester', 'CincinnatiDayton', 'DallasFtWorth', 'HarrisburgScranton', 
           'HartfordSpringfield', 'MiamiFtLauderdale', 'NewOrleansMobile', 'NorthernNewEngland', 'PhoenixTucson',
           'RaleighGreensboro', 'RichmondNorfolk', 'WestTexNewMexico']

greaterregions = ['California', 'GreatLakes', 'Midsouth', 'Northeast', 'Plains', 'SouthCentral', 'Southeast', 'West']

# Funcion de clasificación 
def classify_region(region):
    if region in cities:
        return 'City'
    elif region in regions:
        return 'Region'
    elif region in greaterregions:
        return 'GreaterRegion'
    elif region == 'TotalUS':
        return 'TotalUS'
    elif region == 'SouthCarolina':
        return 'State'
    else:
        return 'Unknown'

# Crear la columna 'region_type'
df['region_type'] = df['region'].apply(classify_region)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    18249 non-null  int64  
 1   Date          18249 non-null  object 
 2   AveragePrice  18249 non-null  float64
 3   Total Volume  18249 non-null  float64
 4   4046          18249 non-null  float64
 5   4225          18249 non-null  float64
 6   4770          18249 non-null  float64
 7   Total Bags    18249 non-null  float64
 8   Small Bags    18249 non-null  float64
 9   Large Bags    18249 non-null  float64
 10  XLarge Bags   18249 non-null  float64
 11  type          18249 non-null  object 
 12  year          18249 non-null  int64  
 13  region        18249 non-null  object 
 14  region_type   18249 non-null  object 
dtypes: float64(9), int64(2), object(4)
memory usage: 2.1+ MB


In [10]:
# Creamos un DataFrame filtrado para las Greater Regions
df_northeast = df[df["region"].isin(["Providence", "Boston", "Albany", "BuffaloRochester", "NorthernNewEngland", "HartfordSpringfield", "HarrisburgScranton", "NewYork", "Pittsburgh", "Philadelphia"])]
df_midsouth = df[df["region"].isin(["Louisville", "RichmondNorfolk", "Charlotte", "RaleighGreensboro", "Roanoke", "BaltimoreWashington", "Nashville"])]
df_southeast = df[df["region"].isin(["MiamiFtLauderdale", "Orlando", "Jacksonville", "SouthCarolina", "Tampa", "Atlanta"])]
df_great_lakes = df[df["region"].isin(["CincinnatiDayton", "GrandRapids", "Indianapolis", "Columbus", "Detroit", "Chicago"])]
df_plains = df[df["region"].isin(["StLouis"])]
df_south_central = df[df["region"].isin(["DallasFtWorth", "NewOrleansMobile", "Houston"])]
df_west = df[df["region"].isin(["Spokane", "Boise", "Seattle", "Portland", "Denver", "WestTexNewMexico", "LasVegas", "PhoenixTucson"])]
df_california = df[df["region"].isin(["SanDiego", "SanFrancisco", "Sacramento", "LosAngeles"])]
#Y otro para Total US
df_totalus = df[df["region"].isin(["TotalUS"])]

In [20]:
# Listas para la clasificación de location

northeast = ["Providence", "Boston", "Albany", "BuffaloRochester", "NorthernNewEngland", "HartfordSpringfield", "HarrisburgScranton", "NewYork", "Pittsburgh", "Philadelphia"]
midsouth = ["Louisville", "RichmondNorfolk", "Charlotte", "RaleighGreensboro", "Roanoke", "BaltimoreWashington", "Nashville"]
southeast = ["MiamiFtLauderdale", "Orlando", "Jacksonville", "SouthCarolina", "Tampa", "Atlanta"]
great_lakes = ["CincinnatiDayton", "GrandRapids", "Indianapolis", "Columbus", "Detroit", "Chicago"]
plains = ["StLouis"]
south_central = ["DallasFtWorth", "NewOrleansMobile", "Houston"]
west = ["Spokane", "Boise", "Seattle", "Portland", "Denver", "WestTexNewMexico", "LasVegas", "PhoenixTucson","West"]
california = ["SanDiego", "SanFrancisco", "Sacramento", "LosAngeles"]

# Funcion de clasificación 
def classify_location(region):
    if region in northeast:
        return 'northeast'
    elif region in midsouth:
        return 'midsouth'
    elif region in southeast:
        return 'southeast'
    elif region in great_lakes:
        return 'great_lakes'
    elif region in plains:
        return 'plains'
    elif region in south_central:
        return 'south_central'
    elif region in west:
        return 'west'
    elif region in california:
        return 'california'
    elif region == 'TotalUS':
        return 'TotalUS'
    else:
        return 'Unknown'

# Crear la columna 'region_type'
df['location'] = df['region'].apply(classify_location)

In [21]:
df.tail(40)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,region_type,location
18209,8,2018-01-28,1.61,7695.89,156.01,859.20,0.00,6680.68,5567.39,1113.29,0.00,organic,2018,Tampa,City,southeast
18210,9,2018-01-21,1.52,6871.05,76.66,407.09,0.00,6387.30,6375.55,11.75,0.00,organic,2018,Tampa,City,southeast
18211,10,2018-01-14,1.53,7238.04,106.98,496.61,0.00,6634.45,6634.45,0.00,0.00,organic,2018,Tampa,City,southeast
18212,11,2018-01-07,1.51,7370.53,42.17,400.58,0.00,6927.78,6921.12,6.66,0.00,organic,2018,Tampa,City,southeast
18213,0,2018-03-25,1.55,1559967.20,121007.94,342853.10,1070.24,1093861.09,902774.79,190941.84,144.46,organic,2018,TotalUS,TotalUS,TotalUS
18214,1,2018-03-18,1.54,1675804.22,170801.85,444949.69,1045.38,1058651.50,837351.85,221129.46,170.19,organic,2018,TotalUS,TotalUS,TotalUS
18215,2,2018-03-11,1.52,1664234.88,129169.72,408763.50,1401.87,1124621.50,944572.50,180049.00,0.00,organic,2018,TotalUS,TotalUS,TotalUS
18216,3,2018-03-04,1.52,1634430.77,142345.03,390129.50,1225.97,1099871.68,831885.50,267818.31,167.87,organic,2018,TotalUS,TotalUS,TotalUS
18217,4,2018-02-25,1.57,1459852.55,122912.97,340374.83,1063.32,994471.50,765056.82,229158.56,256.12,organic,2018,TotalUS,TotalUS,TotalUS
18218,5,2018-02-18,1.52,1814929.97,246515.35,680037.45,1905.46,886241.96,783017.98,103184.01,39.97,organic,2018,TotalUS,TotalUS,TotalUS
